# Exploratory Data Analysis


The objective of this notebook is to explore datasets.
You will need Pandas, a tutorial is available [here](https://pandas.pydata.org/docs/user_guide/10min.html).


Please note that this notebook is inspired from notebooks published by Galiana, Lino. 2023. Python Pour La Data Science. https://doi.org/10.5281/zenodo.8229676.

## Dataset
We consider a dataset gathering information about elections and votes between 2018 and 2022 in the USA. Il also maps economics signals.

In [ ]:
import requests
import pandas as pd

url = 'https://raw.githubusercontent.com/linogaliana/python-datascientist/master/content/modelisation/get_data.py'
r = requests.get(url, allow_redirects=True)
open('getdata.py', 'wb').write(r.content)

import getdata
votes = getdata.create_votes_dataframes()
votes.head(3)

Q1. What is the size of the dataframe? (number of lines and columns)

Print the column names and their types.

Print the statistics of each numerical column (mean, std, quartile, min, max).

In [ ]:
#[STUDENT]


## Q2. What are the different values of the 'winner' variable? Recode this values into numbers 1,2,3,... and store this encoding in a new variable "winner2".

In [ ]:
#[STUDENT


## Descriptive analysis


Q3. Create a dataframe including only the following variables: "winner2", "votes_gop",
          'Unemployment_rate_2021', 'Median_Household_Income_2021',
          'Percent of adults with less than a high school diploma, 2018-22',
          "Percent of adults with a bachelor's degree or higher, 2018-22".
Keep the index "GEOID" as index of your dataframe. For this, use the:
- ```set_index(...)``` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
function

Also, rename the "winner2" variable with "winner" in the new data frame. This can be done with:
- ```df2.rename(columns={'oldColumnName: 'newColumnName}, inplace=True)```

We will refer to the reduced data frame as ```df2```. Display the first 3 columns of the new data.


In [ ]:
#[STUDENT


Q4. Create a frequency tab for each winner value. Build the plot with horizontal bars illustrating this frequency. For this, you can use the:
- ```.plot(...)``` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html
function, which belongs to data frames. To create the frequency tab, the following function is useful:
- ```.value_counts()``` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html


In [ ]:
#[STUDENT

Q5. Let's consider the 'Median_Household_Income_2021' variable. Transform this variable into a categorical one with 5 five labels, and store it in a new column of ```df2``` named "Median_Household_Income_2021_cat". Create the frequency table and the associated bar plot.

For this, you can use the following function:
- ```pd.cut(...)``` https://pandas.pydata.org/docs/reference/api/pandas.cut.html

This function converts *continuous* numerical data into cagtegorical data by establishing *bins*, into which the data fall. These bins can be labelled with the function

In [ ]:
# STUDENT

Q6. Provide descriptive statistics of all variables in the (reduced) dataframe ```df2```.



In [ ]:
#[STUDENT]

Q7. Build a histogram for the variables vote_gop. For this, the following function is useful:
- ```.hist(...)``` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html

In [ ]:
#[STUDENT]
df2['votes_gop'].hist()

Q8. Extract the correlation matrix of ```df2```. Before doing this, remove the "Median_Household_Income_2021_cat" variable from the ```df2``` data. Why do we have to remove this ? (think about the definition of correlation).
- ```.drop(...)``` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

For the correlation matrix, remember the following function:
- ```.corr(...)``` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html

Graph it by using the seaborn package and the following function:
- ```heatmap()``` https://seaborn.pydata.org/generated/seaborn.heatmap.html


Plot a matrix of point clouds of df2 variables with:
- ```pd.plotting.scatter_matrix``` https://pandas.pydata.org/docs/reference/api/pandas.plotting.scatter_matrix.html
Interpret this matrix.

In [ ]:
#[STUDENT]

## Visualization with maps


Q9. Below, we have two blocks of code generating two different graphs (the first map is a choropleth card). They use the same dataset but have different shapes. Comment these graphs and the differences.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# republican : red, democrat : blue
color_dict = {'republican': '#FF0000', 'democrats': '#0000FF'}

fig, ax = plt.subplots(figsize = (12,12))
grouped = votes.groupby('winner')
for key, group in grouped:
    group.plot(ax=ax, column='winner', label=key, color=color_dict[key])
plt.axis('off')

In [ ]:
import plotly
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import numpy as np


centroids = votes.copy()
centroids.geometry = centroids.centroid
centroids['size'] = centroids['CENSUS_2020_POP'] / 10000  # to get reasonable plotable number

color_dict = {"republican": '#FF0000', 'democrats': '#0000FF'}
centroids["winner"] =  np.where(centroids['votes_gop'] > centroids['votes_dem'], 'republican', 'democrats')


centroids['lon'] = centroids['geometry'].x
centroids['lat'] = centroids['geometry'].y
centroids = pd.DataFrame(centroids[["county_name",'lon','lat','winner', 'CENSUS_2020_POP',"state_name"]])
groups = centroids.groupby('winner')

df = centroids.copy()

df['color'] = df['winner'].replace(color_dict)
df['size'] = df['CENSUS_2020_POP']/6000
df['text'] = df['CENSUS_2020_POP'].astype(int).apply(lambda x: '<br>Population: {:,} people'.format(x))
df['hover'] = df['county_name'].astype(str) +  df['state_name'].apply(lambda x: ' ({}) '.format(x)) + df['text']

fig_plotly = go.Figure(
  data=go.Scattergeo(
  locationmode = 'USA-states',
  lon=df["lon"], lat=df["lat"],
  text = df["hover"],
  mode = 'markers',
  marker_color = df["color"],
  marker_size = df['size'],
  hoverinfo="text"
  )
)

fig_plotly.update_traces(
  marker = {'opacity': 0.5, 'line_color': 'rgb(40,40,40)', 'line_width': 0.5, 'sizemode': 'area'}
)

fig_plotly.update_layout(
  title_text = "Reproduction of the \"Acres don't vote, people do\" map <br>(Click legend to toggle traces)",
  showlegend = True,
  geo = {"scope": 'usa', "landcolor": 'rgb(217, 217, 217)'}
)

## Normalization

Q10. Standardize all variables in the ```df2``` dataframe, apart from the ```Median_Household_Income_2021_cat``` variable. Do not overwrite the values of the ```df2``` data frame, create another one. For the scaling, look at:
- ```StandardScaler()``` https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html (also, see lesson slides)

Careful, the function ```scaler.fit_transform``` returns a **numpy** array, and not a pandas data frame. To convert the numpy array back to a data frame, you can use:
- ```pd.DataFrame(x, columns = df2.drop(columns=['Median_Household_Income_2021_cat']).columns)```
This specifies that the columns in x (which have no labels) are taken to be in  the same order as those of ```df2``` (without the ```Median_Household_Income_2021_cat``` column), and are thus labelled in that order in the resulting data frame.


In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# STUDENT


Next, look at the histograms of variable 'the Median_Household_Income_2021 variable' before/after standardization.

In [ ]:
# STUDENT

Q11. Verify that the distribution of "Median_Household_Income_2021" is centered at zero, and that the empirical variance is indeed equal to 1.

In [ ]:
#[STUDENT]

Q12. Create scaler, a function which is built on the first 1000 rows of your ```df2``` DataFrame, with the exception of the ```winner2``` and ```Median_Household_Income_2021_cat``` variables. Hint, the:
- ```.iloc([a:b,c:d])```
function can be used to specify the rows (between ```a``` and ```b```) and the columns (between ```c``` and ```d```) which define the smaller data frame with the first 1000 rows.

Check the mean and standard deviation of each column on these same observations. These are stored in the ```.mean_``` and ```.scale_``` attributes of the scaler.

In [ ]:
# STUDENT


# Outlier detection

Q13. Plot the distribution of each variable in a boxplot and analyze them. Useful function:
- ```sns.boxplot(...)``` https://seaborn.pydata.org/generated/seaborn.boxplot.html

Do you see outliers?

In [ ]:
# STUDENT

Q14. Identify, for each variable, individuals that are not included within the window +/- 3*std. You can remove the following columns: ```'Median_Household_Income_2021_cat', 'winner2','GEOID'```. How many lines would you remove in total?

In [ ]:
# STUDENT



Q15. Let's process over all variables with a library: you can import and use the IsolationForest function from the sklearn.ensemble package.
Change the different parameter values to identify their impact.
Do you obtain results different from the analysis of single variables.

In [ ]:
# prompt: do it

from sklearn.ensemble import IsolationForest

# Assuming df2 is already defined as in the previous code.

# Create an IsolationForest model
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=42)

# Fit the model to your data
model.fit(df2.drop(columns=['Median_Household_Income_2021_cat', 'winner']))

# Get the prediction of the model
df2['anomaly'] = model.predict(df2.drop(columns=['Median_Household_Income_2021_cat', 'winner']))

# Print the number of anomalies detected by the model
print(f"Number of anomalies detected: {df2[df2['anomaly'] == -1].shape[0]}")


Q16. Display in a scatter plot variables 'votes_gop' and 'Unemployment_rate_2021' and color points according whether they are outliers or not. Interpret.
You can change pairs of variables.

In [ ]:
# STUDENT


Q17. Display for all pairs of variables using pairplot from seaborn.


In [ ]:
#[STUDENT]